In [49]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [50]:
model = keras.models.load_model('mnist_cnn.h5')

In [51]:
def get_img_contour_thresh(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (35, 35), 0)
    ret, thresh1 = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    return contours, thresh1

In [59]:
sample = cv2.imread('test1.jpg')
sample = cv2.cvtColor(sample, cv2.COLOR_RGB2GRAY)
sample = cv2.resize(sample,(28,28))
cap = cv2.VideoCapture(0)
cap.set(3, 1000)
cap.set(4, 600)
x, y, w, h = 400, 240, 200, 200
while (cap.isOpened()):
    ret, img = cap.read()
    frame = img[y:y+h, x:x+w]
    contours, thresh = get_img_contour_thresh(frame)
    ans1 = ''
    if len(contours) > 0:
        contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(contour) > 2500:
            x1, y1, w1, h1 = cv2.boundingRect(contour)
            newImage = thresh[y1:y1 + h1, x1:x1 + w1]
            newImage = cv2.resize(newImage, (28, 28))
            newImage = np.array(newImage)
            newImage = newImage.flatten()
            newImage = newImage.reshape(1, 28,28,1)
            ans1 = model.predict_classes(newImage)
            ans1 = ans1[0]
    img = cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    img = cv2.putText(img, 'Predicted:' + str(ans1),(50,50),cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2) 
    cv2.imshow("Frame", img)
    cv2.imshow("Cropped", frame)
    cv2.imshow("Threshold", thresh)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [36]:
sample = np.array(sample)
sample = np.reshape(sample,(1,28,28,1))
predicted = model.predict_classes(sample)

In [37]:
predicted[0]

5